In [1]:
from bs4 import BeautifulSoup 
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# from sites import NASA_Mars_News_Site, JPL_Featured_Space_Image, Mars_Facts, USGS_Astrogeology_site

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [/Users/joeyriebel/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [3]:
# NASA Url
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [4]:
# HTML Object
html = browser.html
#Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

In [5]:

gallery = soup.find(class_='grid_gallery')
news_title = gallery.find(class_="content_title").text
news_p = gallery.find(class_='article_teaser_body').text
news_p

'NASA’s newest rover recorded audio of itself crunching over the surface of the Red Planet, adding a whole new dimension to Mars exploration.'

In [6]:
# Use splinter to brouse through page
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [7]:
# Handle search

browser.find_by_tag('input').first.fill('Featured Mars')

In [8]:
# Handle Sortby
browser.find_by_id('searchHelpers_sortBy').first.select('latestDate')

In [9]:
results = browser.find_by_id('SearchListingPageResults')

In [11]:
results.links.find_by_partial_href('images').click()
browser.find_by_text('Download JPG ').click

<bound method WebDriverElement.click of <splinter.driver.webdriver.WebDriverElement object at 0x7f97330f2df0>>

In [12]:
featured_image_url = browser.url
featured_image_url

'https://www.jpl.nasa.gov/images/noachis-terra-false-color-11'

In [13]:
url = 'https://space-facts.com/mars/'
browser.visit(url)


In [14]:
df=pd.read_html('https://space-facts.com/mars/')[0]

In [15]:
# Rename Columns
df.columns=['Description','Mars']
df

,Description,Mars
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [16]:
html_table= df.to_html(index=False)
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Description</th>\n      <th>Mars</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

In [17]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [18]:
hemisphere_images_url=[]
links=browser.find_by_css('a.product-item img')
for i in range(4):
    hemisphere={}
    browser.find_by_css('a.product-item img')[i].click()
    html=browser.html
    image_soup=BeautifulSoup(html,'html.parser')
    image=image_soup.find_all('div',class_='downloads')[0].find('a')['href']
    title=image_soup.find_all('h2',class_='title')[0].text
    hemisphere['title']=title
    hemisphere['image_url']=image
    hemisphere_images_url.append(hemisphere)
    time.sleep(0.5)
    browser.back()

In [19]:
hemisphere_images_url

[{'title': 'Cerberus Hemisphere Enhanced',
  'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [20]:
def quit_browser():
    browser.quit()

In [24]:
mars_dict={
    "Title":news_title,
    "Paragraph":news_p,
    "FeaturedImage": featured_image_url,
    "MarsFacts": html_table,
    "HemisphereImages": hemisphere_images_url
}

print(mars_dict)

{'Title': 'Another First: Perseverance Captures the Sounds of Driving on Mars', 'Paragraph': 'NASA’s newest rover recorded audio of itself crunching over the surface of the Red Planet, adding a whole new dimension to Mars exploration.', 'FeaturedImage': 'https://www.jpl.nasa.gov/images/noachis-terra-false-color-11', 'MarsFacts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Description</th>\n      <th>Mars</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)<

In [25]:
mars_dict["HemisphereImages"][0]["title"]

'Cerberus Hemisphere Enhanced'

In [26]:
#Dependencies
import pymongo

In [27]:
#Innitialize PyMongo
conn="mongodb://localhost:27017"
client=pymongo.MongoClient(conn)

In [28]:
#Define database and collecction
db=client.nasa_db
collection=db.items